In [2]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
import os
import random
import shutil
import math
import cv2
import tensorflow.keras.models
import tensorflow.keras.layers
import tensorflow.keras.applications
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16
import keras
from tensorflow.keras.utils import serialize_keras_object

In [3]:
def build_model(): 
    input_layer = Input(shape=(120,120,3))
    
    vgg = VGG16(include_top=False)(input_layer)
    #Model de classificació
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1) #relu == funció que determina la classe; 
    class2 = Dense(3, activation='sigmoid')(class1) # sigmoid == funció que determina la presició de la classe

    # sigmoid = f(x) = 1/(1+e^-x)
    
    #Model de localització de coordenades
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)
    
    detector = Model(inputs=input_layer, outputs=[class2, regress2])
    return detector

In [4]:
detector = build_model()

In [5]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001) #li introduïm el decay que hem calculat a l'optimitzador

In [6]:
def localization_loss(y_true, yhat):#primer valor: coordenades reals, segon valor: coordenades previstes     
    y_true = tf.reshape(y_true, (5, 4))
    
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2])) #diferència dels dos primers valors de cada fila de la matriu
                  
    h_true = y_true[:,3] - y_true[:,1] #quarta columna d'una matriu - segona columna
    w_true = y_true[:,2] - y_true[:,0] #tercera columna - primera

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    '''
    delta_size = suma dels quadrats de les diferències entre les dimensions originals 
    i les dimensions reconstruïdes de l'imatge.
    '''
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    return delta_coord + delta_size

In [7]:
classloss = tf.keras.losses.CategoricalCrossentropy() #model que fa una classificació binaria 
regressloss = localization_loss #model que acabem de crear

In [8]:
class Detector(Model):
    def __init__(self, fruita, **kwargs):
        super().__init__(**kwargs)
        self.base_model = fruita

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt

    def train_step(self, batch, **kwargs):
        X, y = batch

        with tf.GradientTape() as tape:
            classes, coords = self.base_model(X, training=True)
            batch_classloss = self.closs(tf.reshape(y[0], (5, 3)), classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)

            total_loss = batch_localizationloss + 0.5 * batch_classloss

        grad = tape.gradient(total_loss, self.base_model.trainable_variables)
        self.opt.apply_gradients(zip(grad, self.base_model.trainable_variables))

        return {"total_loss": total_loss, "class_loss": batch_classloss, "regress_loss": batch_localizationloss}

    def test_step(self, batch, **kwargs):
        X, y = batch

        classes, coords = self.base_model(X, training=False)

        batch_classloss = self.closs(tf.reshape(y[0], (5, 3)), classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss + 0.5 * batch_classloss

        return {"total_loss": total_loss, "class_loss": batch_classloss, "regress_loss": batch_localizationloss}

    def call(self, X, **kwargs):
        return self.base_model(X, **kwargs)
    
    def get_config(self):
        # Get the base configuration
        base_config = super().get_config()
        
        # Serialize the "fruita" model and store it in the configuration
        fruita_config = serialize_keras_object(self.base_model)
        
        # Construct the complete configuration dictionary
        config = {
            "fruita": fruita_config,
        }
        
        # Merge the base configuration and the custom configuration
        return {**base_config, **config}

In [9]:
model = Detector(detector)
model.build(input_shape=(None, 120, 120, 3))

In [10]:
model.load_weights('detector_fruites_weights.h5')

webcam

In [ ]:
import cv2

cap = cv2.VideoCapture(1)
cv2.namedWindow("Object Detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Object Detection", 800, 800)  # Adjust the size as needed

while cap.isOpened():
    ret, frame = cap.read()

    # Mida del display
    display_frame = cv2.resize(frame, (960, 960))

    # Reduir la mida de la imatge per a la introducció a la xarxa neuronal
    input_frame = cv2.resize(frame, (120, 120))

    # Convertir la imatge a un array de 3 dimensions amb valors entre 0 i 1 (necessari per a la xarxa neuronal)
    input_data = input_frame.reshape((1, 120, 120, 3)) / 255.0  

    # Executa la predicció
    predictions = model.predict(input_data)

    # Bucle per a les diferents dades predites
    for prediction in predictions:
        confidences_list = [] # les col·loquem en una llista

        for prediction in predictions:
            confidence_array = prediction[0]
            confidences_list.append(confidence_array)

        #divisió i preparació de les dades
        prob_poma = float(confidences_list[0][0])
        prob_pera = confidences_list[0][1]
        prob_mandarina = confidences_list[0][2]
        x1, r = math.modf(confidences_list[1][0])
        x2, r = math.modf(confidences_list[1][1])
        y1, r = math.modf(confidences_list[1][2])
        y2, r = math.modf(confidences_list[1][3])
        print(prob_poma, prob_pera, prob_mandarina)
    
        
        if prob_poma > 0.8: #si la probabilitat de que sigui una poma és més gran que 80%
            cv2.rectangle(display_frame, (int(x1*940), int(x2*940)), (int(y1*940), int(y2*940)), (0, 255, 0), 2)
            label = f'Probabilitat poma: {prob_poma:.2f}'
            cv2.putText(display_frame, label, (int((x1*940)+20), int((x2*940)+20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if prob_pera > 0.6: #si la probabilitat de que sigui una pera és més gran que 60%
            cv2.rectangle(display_frame, (int(x1*940), int(x2*940)), (int(y1*940), int(y2*940)), (0, 255, 255), 2)
            label = f'Probabilitat pera: {prob_pera:.2f}'
            cv2.putText(display_frame, label, (int((x1*940)+20), int((x2*940)+20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
            
        if prob_mandarina > 0.7: #si la probabilitat de que sigui una mandarina és més gran que 70%
            cv2.rectangle(display_frame, (int(x1*940), int(x2*940)), (int(y1*940), int(y2*940)), (255, 102, 0), 2)
            label = f'Probabilitat mandarina: {prob_mandarina:.2f}'
            cv2.putText(display_frame, label, (int((x1*940)+20), int((x2*940)+20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 102, 0), 2)

    # Mostrar la imatge
    cv2.imshow("Detecció de Fruites", display_frame)

    # Tancar la finestra si es prem la tecla q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# tanca les finestres
cap.release()
cv2.destroyAllWindows()
